In [1]:
"""
Script to render the asset pricing table
"""

import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from scipy.stats import ttest_1samp
from regtabletotext import prettify_result
import warnings
warnings.filterwarnings("ignore")

from environ.constants import (
    DEPENDENT_VARIABLES,
    DEPENDENT_VARIABLES_ASSETPRICING,
    PROCESSED_DATA_PATH,
    STABLE_DICT,
    ALL_NAMING_DICT,
    TABLE_PATH,
)
from environ.process.asset_pricing.double_sorting import calculate_period_return
from environ.process.asset_pricing.assetpricing_functions import (
    reg_fama_macbeth, clean_weekly_panel, univariate_sort, univariate_sort_table, double_sort, double_sort_table, get_dominance_portfolios, significance_stars
    )
                                                                  

In [2]:
# load factors
ff3 = pd.read_csv(PROCESSED_DATA_PATH/"FF3.csv") 
ltw3 = pd.read_csv(PROCESSED_DATA_PATH/"LTW3.csv")

# load the regression panel dataset
reg_panel = pd.read_pickle(
    PROCESSED_DATA_PATH / "panel_main.pickle.zip", compression="zip"
)

# stable non-stable info dict
stable_nonstable_info = {
    "stablecoin": reg_panel[reg_panel["Token"].isin(STABLE_DICT.keys())],
    "non-stablecoin": reg_panel[~reg_panel["Token"].isin(STABLE_DICT.keys())],
    "all": reg_panel,
}

# How are returns aggregated for each portfolio
ret_agg = 'mean'

# DEPENDENT_VARIABLES_ASSETPRICING =['volume_ultimate_share']  #,'volume_in_share' , 'volume_out_share']
# ,'eigen_centrality_undirected','total_eigen_centrality_undirected','Volume_share']

### Univariate sorting

In [3]:
for dom_variable in DEPENDENT_VARIABLES_ASSETPRICING:
    for is_boom in [-1]:
        quantiles, separate_zero_value = [0,0.3,0.7,1], False #[0,0.9,0.95,1] #
        df_panel = clean_weekly_panel(reg_panel, is_stablecoin = 0, is_boom = is_boom)

        # Substract risk free rate
        df_panel = pd.merge(df_panel,ff3, on='WeekYear')
        df_panel['ret'] = df_panel['ret']-df_panel['RF']

        df_panel = univariate_sort(df_panel, dom_variable, quantiles=quantiles, separate_zero_value=separate_zero_value)
        summary_table = univariate_sort_table(df_panel, ret_agg = ret_agg)
    
        if is_boom == 1:
            boom_str = " boom"
        elif is_boom == 0:
            boom_str = " bust"
        else:
            boom_str = " alltime"
        summary_table = summary_table.style.set_caption(dom_variable+' '+boom_str)
        display(summary_table)

,P1,P2,P3,P3-P1
Mean,0.019116,0.008810,0.005646,-0.013470
t-Stat,1.609453,0.792199,0.604530,-2.033038
StdDev,0.137491,0.128728,0.108120,0.076695
Sharpe,1.003976,0.494173,0.377105,-1.268208


,P1,P2,P3,P3-P1
Mean,0.018127,0.013512,0.003537,-0.014590
t-Stat,1.556163,1.123259,0.387126,-2.169530
StdDev,0.134843,0.139249,0.105756,0.077850
Sharpe,0.970734,0.700688,0.241489,-1.353351


,P1,P2,P3,P3-P1
Mean,0.018620,0.010595,0.005894,-0.012726
t-Stat,1.593704,0.914212,0.634808,-1.937609
StdDev,0.135246,0.134158,0.107483,0.076027
Sharpe,0.994152,0.570285,0.395993,-1.208679


,P1,P2,P3,P3-P1
Mean,0.019404,0.009461,0.003877,-0.015527
t-Stat,1.628990,0.844091,0.429540,-2.283541
StdDev,0.137884,0.129743,0.104479,0.078708
Sharpe,1.016163,0.526543,0.267947,-1.424472


,P1,P2,P3,P3-P1
Mean,0.019214,0.011485,0.003661,-0.015553
t-Stat,1.612889,1.028116,0.402107,-2.273371
StdDev,0.137897,0.129316,0.105383,0.079194
Sharpe,1.006120,0.641338,0.250834,-1.418127


,P1,P2,P3,P3-P1
Mean,0.019620,0.006472,0.005046,-0.014573
t-Stat,1.647616,0.578644,0.549758,-2.122018
StdDev,0.137843,0.129467,0.106259,0.079498
Sharpe,1.027782,0.360958,0.342939,-1.323714


,P1,P2,P3,P3-P1
Mean,0.011256,0.006483,0.055262,0.044005
t-Stat,1.487584,0.497258,3.565519,3.816618
StdDev,0.087594,0.150923,0.179413,0.133468
Sharpe,0.927954,0.310189,2.224169,2.380804


In [6]:

        
def dataframe_to_latex_rows(df, column_format="lcccc"):
    """
    Returns a string containing only the data rows from df.to_latex(),
    with no \\toprule, \\bottomrule, or \\begin{tabular} lines.
    """
    raw_latex = df.to_latex(
        index=True,
        header=False,     # We will write the column header manually
        float_format="%.4f",
        column_format=column_format,
        escape=False      # allow us to keep LaTeX symbols in row labels
    )

    lines = raw_latex.splitlines()
    stripped = []
    for line in lines:
        if any(token in line for token in [
            r"\toprule", r"\bottomrule", r"\midrule",
            "tabular", "begin{", "end{"
        ]):
            # Skip these lines
            continue
        stripped.append(line)
    return "\n".join(stripped)

with open("univariate_sort.tex", "w") as f:
    # Table setup
    f.write(r"\begin{table}[ht]" + "\n")
    f.write(r"\centering" + "\n")
    f.write(r"\caption{Univariate Sort on dominance measures}" + "\n")
    f.write(r"\label{tab:univariate_sort}" + "\n\n")

    # Adjust the column format to fit your columns (7 columns => lcccccc).
    # Here we have 5 columns: (row label) + 4 data columns
    # If you want 6 or 7 columns, adapt accordingly.
    f.write(r"\begin{tabular}{lcccc}" + "\n")
    f.write(r"\toprule" + "\n")
    f.write(r"       & L    & 2    & H    & H--L \\" + "\n")  
    f.write(r"\midrule" + "\n\n")

    quantiles, separate_zero_value = [0,0.3,0.7,1], False
    for dom_variable in DEPENDENT_VARIABLES_ASSETPRICING:
        df_panel = clean_weekly_panel(reg_panel, is_stablecoin = 0, is_boom = is_boom)
        df_panel = pd.merge(df_panel,ff3, on='WeekYear')
        df_panel['ret'] = df_panel['ret']-df_panel['RF']
        df_panel = univariate_sort(df_panel, dom_variable, quantiles=quantiles, separate_zero_value=separate_zero_value)
        summary_table = univariate_sort_table(df_panel, ret_agg = ret_agg)

        if dom_variable == DEPENDENT_VARIABLES_ASSETPRICING[-1]:
            # last panel
            f.write(r"\multicolumn{5}{c}{\textbf{Panel D: ME}} \\" + "\n")
            f.write(r"\midrule" + "\n")
            f.write(dataframe_to_latex_rows(summary_table, column_format="lcccc"))
            f.write("\n" + r"\bottomrule" + "\n")

            f.write(r"\end{tabular}" + "\n")
            f.write(r"\end{table}" + "\n")
        else:
            # mid panel
            f.write(r"\multicolumn{5}{c}{\textbf{Panel A: {dom_variable}}} \\" + "\n")
            f.write(r"\midrule" + "\n")
            f.write(dataframe_to_latex_rows(summary_table, column_format="lcccc"))
            f.write("\n" + r"\midrule" + "\n\n")

In [4]:
stop

NameError: name 'stop' is not defined

In [ ]:
df_panel.portfolio.value_counts()

portfolio
P2    2914
P1    2279
P3    2258
Name: count, dtype: int64

In [ ]:
test = clean_weekly_panel(reg_panel, is_stablecoin = 0, is_boom = -1)
test.describe()

,ret,volume_ultimate_share,eigen_centrality_undirected,vol_inter_full_len_share,betweenness_centrality_volume,betweenness_centrality_count,total_eigen_centrality_undirected,Volume_share,volume_in_share,volume_out_share,mcap,amihud,ret_lead_1
count,31717.000000,31717.000000,31717.000000,31717.000000,31717.000000,31717.000000,31717.000000,31717.000000,31717.000000,31717.000000,3.171700e+04,7.451000e+03,31717.000000
mean,0.002189,0.004126,0.009530,0.004207,0.001994,0.002726,0.009403,0.004133,0.004133,0.004133,1.789099e+09,9.693595e-06,0.004515
std,0.235466,0.033640,0.063393,0.047683,0.033558,0.041681,0.063652,0.034300,0.034246,0.034371,1.823303e+10,2.443119e-04,0.238711
min,-0.800279,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,-0.800279
25%,-0.114990,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.019431e+06,8.541736e-09,-0.114536
50%,-0.011518,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.042572e+07,3.372519e-08,-0.011403
75%,0.070594,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.924121e+08,1.145171e-07,0.072440
max,4.410082,0.467854,0.705255,0.993955,0.979208,0.977260,0.706829,0.495497,0.505160,0.514906,5.560275e+11,1.283550e-02,4.410082


In [ ]:
df_panel.groupby('portfolio')['mcap'].median()

portfolio
P1    1.438855e+09
P2    1.761111e+08
P3    5.282951e+07
Name: mcap, dtype: float64

### Double sort

In [ ]:
for secondary_variable in ['mcap']:
    for dom_variable in DEPENDENT_VARIABLES_ASSETPRICING:
        for is_boom in [-1]:
            quantiles, separate_zero_value = [0,0.3,0.7,1], False #[0,0.25,0.5,0.75,1] 
            df_panel = clean_weekly_panel(reg_panel, is_stablecoin = 0, is_boom = is_boom)
            df_panel = pd.merge(df_panel,ff3, on='WeekYear')
            df_panel= double_sort(df_panel, dom_variable, secondary_variable=secondary_variable, quantiles=quantiles, separate_zero_value=separate_zero_value)
            summary_table = double_sort_table(df_panel, ret_agg="mean")
            if is_boom == 1:
                boom_str = " boom"
            elif is_boom == 0:
                boom_str = " bust"
            else:
                boom_str = "alltime"
            summary_table = summary_table.style.set_caption(dom_variable +' '+ boom_str)
            display(summary_table)

primary_portfolio,P1,P2,P3
secondary_portfolio,,,
Q1,0.011587,0.010040,-0.004693
Q2,-0.000819,-0.005455,0.003763
Q3,0.002562,0.016821,0.011601


primary_portfolio,P1,P2,P3
secondary_portfolio,,,
Q1,0.010969,0.022947,-0.012714
Q2,-0.000836,-0.009610,0.007170
Q3,0.002339,0.020142,0.010706


primary_portfolio,P1,P2,P3
secondary_portfolio,,,
Q1,0.011272,0.016070,-0.006985
Q2,-0.000852,-0.007020,0.005226
Q3,0.002374,0.017252,0.012001


primary_portfolio,P1,P2,P3
secondary_portfolio,,,
Q1,0.011711,0.013051,-0.009637
Q2,-0.000998,-0.008746,0.002585
Q3,0.003001,0.019611,0.011509


primary_portfolio,P1,P2,P3
secondary_portfolio,,,
Q1,0.011538,0.017081,-0.009878
Q2,-0.001208,-0.006194,0.002578
Q3,0.002761,0.018928,0.010793


primary_portfolio,P1,P2,P3
secondary_portfolio,,,
Q1,0.011931,0.007096,-0.007573
Q2,-0.000577,-0.008716,0.003586
Q3,0.002956,0.016921,0.011762


primary_portfolio,P1,P2,P3
secondary_portfolio,,,
Q1,0.000786,0.001373,0.083934
Q2,0.013788,-0.000924,0.031249
Q3,0.014511,0.010425,0.034312


# Factor testing

In [ ]:

factor_models = ["MKT + SMB + HML", "CMKT", "CMKT + CMOM + CSIZE"]
is_boom = -1

for factor_model in factor_models:
    for dom_variable in DEPENDENT_VARIABLES_ASSETPRICING:
        for is_boom in [-1]:
            # 1. Prepare your data
            quantiles, separate_zero_value = [0, 0.3, 0.7, 1], False
            df_panel = clean_weekly_panel(reg_panel, is_stablecoin=0, is_boom=is_boom)
            df_panel = univariate_sort(
                df_panel, dom_variable, quantiles, separate_zero_value=separate_zero_value
            )
            dominance_portfolios = get_dominance_portfolios(df_panel)
            portfolios = list(dominance_portfolios.columns)

            # 2. Merge all factors into a single DataFrame
            factors_data = pd.merge(dominance_portfolios, ff3, on=["WeekYear"], how="left")
            factors_data = pd.merge(factors_data, ltw3, on=["WeekYear"], how="left")
            factors_data = factors_data.dropna()

            # 3. Build a list of factor names from the formula (plus "alpha")
            #    Example: factor_model="MKT + SMB + HML" => ["MKT", "SMB", "HML"]
            #    We'll store "alpha" and then each factor, plus a matching "_t" row for t-stats
            raw_factors = factor_model.replace(" ", "").split("+")
            factor_names = ["alpha"] + raw_factors  # "alpha" is the renamed Intercept
            row_list = []
            for f in factor_names:
                row_list.append(f)      # e.g. "alpha", "MKT", "SMB", ...
                row_list.append(" ")  # e.g. "alpha_t", "MKT_t", ...

            # Finally, add R-squared and N at the bottom
            row_list += ["R-squared", "N"]
            final_table = pd.DataFrame(index=row_list, columns=portfolios)

            # 4. Run a separate regression for each portfolio
            for p in portfolios:
                formula = f"{p} ~ {factor_model}"

                # Use Newey–West (HAC) standard errors
                model = smf.ols(formula=formula, data=factors_data).fit(
                    cov_type="HAC", cov_kwds={"maxlags": 4}
                )

                # Extract estimates, t-stats, p-values
                coefs = model.params.copy()
                tvals = model.tvalues.copy()
                pvals = model.pvalues.copy()

                # Rename "Intercept" to "alpha"
                if "Intercept" in coefs.index:
                    coefs.rename({"Intercept": "alpha"}, inplace=True)
                    tvals.rename({"Intercept": "alpha"}, inplace=True)
                    pvals.rename({"Intercept": "alpha"}, inplace=True)

                # Fill each factor row with the coefficient and the next row with the t-stat
                for f in factor_names:
                    # If the factor is in the model (sometimes a factor might be missing),
                    # then fill the table; otherwise leave as NaN
                    if f in coefs.index:
                        star = significance_stars(pvals[f])
                        # Row for coefficient (with stars)
                        final_table.loc[f, p] = f"{coefs[f]:.4f}{star}"
                        # Row for t-stat
                        final_table.loc[" ", p] = f"({tvals[f]:.2f})"
                    else:
                        # If factor not found in the regression, fill with blanks or zeros
                        final_table.loc[f, p] = ""
                        final_table.loc[" ", p] = ""

                # Fill in R-squared and # obs
                final_table.loc["R-squared", p] = f"{model.rsquared:.3f}"
                final_table.loc["N", p]         = f"{int(model.nobs)}"

            # 5. Print or export the final table
            print(f"== Results for {dom_variable} | Model: {factor_model} ")
            print(final_table.to_latex())
            # final_table.to_latex('panelA.tex', index=True, header=False, column_format='lrrrr', 
            # bold_rows=True).replace('\\toprule\n', '').replace('\\bottomrule\n', '')



== Results for volume_ultimate_share | Model: MKT + SMB + HML 
\begin{tabular}{lllll}
\toprule
 & P1 & P2 & P3 & P3-P1 \\
\midrule
alpha & 0.0191 & 0.0104 & 0.0061 & 0.0061 \\
  & (-0.84) & (-1.18) & (-0.70) & (-0.70) \\
MKT & 0.4225 & 0.1291 & 0.2958 & 0.2958 \\
  & (-0.84) & (-1.18) & (-0.70) & (-0.70) \\
SMB & 0.3808 & -0.0246 & -0.3336 & -0.3336 \\
  & (-0.84) & (-1.18) & (-0.70) & (-0.70) \\
HML & -0.3910 & -0.5681 & -0.3066 & -0.3066 \\
  & (-0.84) & (-1.18) & (-0.70) & (-0.70) \\
R-squared & 0.022 & 0.017 & 0.014 & 0.014 \\
N & 134 & 134 & 134 & 134 \\
\bottomrule
\end{tabular}

== Results for eigen_centrality_undirected | Model: MKT + SMB + HML 
\begin{tabular}{lllll}
\toprule
 & P1 & P2 & P3 & P3-P1 \\
\midrule
alpha & 0.0179 & 0.0162 & 0.0035 & 0.0035 \\
  & (-0.76) & (-1.49) & (-0.47) & (-0.47) \\
MKT & 0.4242 & 0.0474 & 0.3478 & 0.3478 \\
  & (-0.76) & (-1.49) & (-0.47) & (-0.47) \\
SMB & 0.4140 & -0.1924 & -0.2123 & -0.2123 \\
  & (-0.76) & (-1.49) & (-0.47) & (-0.47) \\
H

### FAMA MCBETH 

In [ ]:
factor_models = ["CMKT+CMOM+CSIZE"]
is_boom = -1

for factor_model in factor_models:
    for dom_variable in DEPENDENT_VARIABLES_ASSETPRICING:
        quantiles, separate_zero_value = [0, 0.3, 0.7, 1], False
        df_panel = clean_weekly_panel(reg_panel, is_stablecoin=0, is_boom=is_boom)
        df_panel = univariate_sort(df_panel, dom_variable, quantiles, separate_zero_value=separate_zero_value)
        dominance_factor = get_dominance_portfolios(df_panel)
        dominance_factor.rename(columns={dominance_factor.columns[-1]: "CDOM"}, inplace=True)
        # Get the test assets
        assets_panel = clean_weekly_panel(reg_panel, is_stablecoin=0, is_boom=-1)
        # Merge all factors
        data_fama_macbeth = pd.merge(dominance_factor, ff3, on=["WeekYear"], how="left")
        data_fama_macbeth = pd.merge(data_fama_macbeth, ltw3, on=["WeekYear"], how="left")
        # Merge factors with returns
        data_fama_macbeth = pd.merge(data_fama_macbeth, assets_panel, on=["WeekYear"], how="left")
        data_fama_macbeth = data_fama_macbeth.dropna()

        # Run the Fama–MacBeth regression
        data_fama_macbeth['excess_ret'] = data_fama_macbeth['ret'] - data_fama_macbeth['RF']
        fama_macbeth_results = reg_fama_macbeth(data_fama_macbeth, formula="excess_ret ~ CMKT + CMOM + CSIZE + CDOM")

        # Set caption based on is_boom value
        if is_boom == 1:
            boom_str = " boom"
        elif is_boom == 0:
            boom_str = " bust"
        else:
            boom_str = " alltime"
        caption_str = dom_variable + boom_str

        # Convert regression results DataFrame to a LaTeX table string
        # latex_table = fama_macbeth_results.to_latex(index=False)

        # # Wrap the table with a caption and table environment
        # latex_table = (
        #     "\\begin{table}[ht]\n"
        #     "\\centering\n"
        #     "\\caption{" + caption_str + "}\n"
        #     + latex_table +
        #     "\n\\end{table}"
        # )

        # # Print the LaTeX table code
        # print(latex_table)
        file_name = (
                TABLE_PATH
                / "assetpricing"
                / f"assetpricing_famamacbeth_{dom_variable}_{factor_model}"
            )

        fama_macbeth_results.to_latex(
            f"{file_name}.tex",
            index=True,
            escape=False,
        )


In [ ]:
"""
Script to render the table of Fama Macbeth.
"""

from pathlib import Path
import pandas as pd
import numpy as np
from scipy.stats import ttest_1samp
from environ.constants import (
    ALL_NAMING_DICT,
    DEPENDENT_VARIABLES_ASSETPRICING,
    PROCESSED_DATA_PATH,
    TABLE_PATH,
)
from environ.process.asset_pricing.assetpricing_functions import (
    clean_weekly_panel,
    univariate_sort,
    get_dominance_portfolios,
    reg_fama_macbeth,
)


if __name__ == "__main__":
    # compute means for portfolio returns (can change to median)
    ret_agg = "mean"
    is_boom = -1
    # load the regression panel dataset
    reg_panel = pd.read_pickle(
        PROCESSED_DATA_PATH / "panel_main.pickle.zip", compression="zip"
    )
    # load factors
    ff3 = pd.read_csv(PROCESSED_DATA_PATH / "FF3.csv")
    ltw3 = pd.read_csv(PROCESSED_DATA_PATH / "LTW3.csv")
    for dom_variable in DEPENDENT_VARIABLES_ASSETPRICING[:1]:
        quantiles, separate_zero_value = [0, 0.3, 0.7, 1], False
        df_panel = clean_weekly_panel(reg_panel, is_stablecoin=0, is_boom=is_boom)
        df_panel = univariate_sort(
            df_panel, dom_variable, quantiles, separate_zero_value=separate_zero_value
        )
        dominance_factor = get_dominance_portfolios(df_panel)
        dominance_factor.rename(
            columns={dominance_factor.columns[-1]: "CDOM"}, inplace=True
        )
        # Get the test assets
        assets_panel = clean_weekly_panel(reg_panel, is_stablecoin=0, is_boom=-1)
        # Merge all factors
        data_fama_macbeth = pd.merge(dominance_factor, ff3, on=["WeekYear"], how="left")
        data_fama_macbeth = pd.merge(
            data_fama_macbeth, ltw3, on=["WeekYear"], how="left"
        )
        # Merge factors with returns
        data_fama_macbeth = pd.merge(
            data_fama_macbeth, assets_panel, on=["WeekYear"], how="left"
        )
        data_fama_macbeth = data_fama_macbeth.dropna()

        # Run the Fama–MacBeth regression
        data_fama_macbeth["excess_ret"] = (
            data_fama_macbeth["ret"] - data_fama_macbeth["RF"]
        )
        fama_macbeth_results = reg_fama_macbeth(
            data_fama_macbeth, formula="excess_ret ~ CMKT + CMOM + CSIZE + CDOM"
        )
        fama_macbeth_results = fama_macbeth_results.round(3)
        fama_macbeth_results.drop("t_stat", axis=1, inplace=True)
        fama_macbeth_results.rename(
            columns={"factor":"Factor", "risk_premium":"Risk Premium", "t_stat_NW":r"\emph{t}"}, inplace=True
        )
        file_name = (
            TABLE_PATH / "assetpricing" / f"assetpricing_famamacbeth_{dom_variable}"
        )

        fama_macbeth_results.to_latex(
            f"{file_name}.tex",
            index=True,
            escape=False,
        )


In [14]:
import pandas as pd

# -----------------------------------------------------------------------------
# 1. Create sample data
#    (In your real code, just replace these with your existing DataFrames)
# -----------------------------------------------------------------------------
dfA = pd.DataFrame({
    ' ': ['α', 'β', 'MKT', 'R²'],
    'L':   [-0.88, 1.42, 13.94, 0.82],
    '2':   [-0.12, 0.91, 10.11, 0.79],
    '3':   [0.49, 0.93, 10.16, 0.84],
    '4':   [1.33, 0.96,  9.33, 0.85],
    '5':   [2.19, 0.97,  8.47, 0.86],
    'H':   [3.19, 0.94,  6.59, 0.82],
    'H-L': [4.07, -0.48, -7.35, 0.09]
})

dfB = pd.DataFrame({
    ' ':  ['α', 'β', 'MKT', 'SMB', 'HML', 'R²'],
    'L':   [-1.82, 0.67, 28.85,  0.15,  0.40, 0.85],
    '2':   [ -1.54, 0.82, 24.70,  0.11,  0.32, 0.81],
    '3':   [ -0.22, 0.91, 24.61,  0.20,  0.15, 0.80],
    '4':   [ 0.97,  0.96, 25.29,  0.31, -0.07, 0.82],
    '5':   [ 1.43,  0.98, 25.50,  0.29, -0.14, 0.84],
    'H':   [ 2.59,  1.03, 24.57,  0.41, -0.19, 0.85],
    'H-L': [ 4.41,  0.36, -4.28,  0.26, -0.59, 0.12]
})

dfC = pd.DataFrame({
    ' ':   ['α', 'β', 'MKT', 'SMB', 'HML', 'UMD', 'R²'],
    'L':   [-0.89, 0.68, 27.60, 0.16,  0.41,  0.20, 0.86],
    '2':   [-0.23, 0.83, 24.55, 0.15,  0.33,  0.19, 0.81],
    '3':   [ 0.48, 0.90, 24.70, 0.15,  0.10,  0.28, 0.81],
    '4':   [ 1.31, 0.97, 25.45, 0.32, -0.06,  0.16, 0.83],
    '5':   [ 2.20, 1.00, 25.51, 0.29, -0.14, -0.02, 0.84],
    'H':   [ 3.10, 1.04, 24.55, 0.40, -0.18, -0.09, 0.85],
    'H-L': [ 3.99, 0.36, -3.05, 0.24, -0.59, -0.29, 0.14]
})

# -----------------------------------------------------------------------------
# 2. Helper function to extract the body of the LaTeX table (without \begin{tabular} etc.)
# -----------------------------------------------------------------------------
def df_to_latex_body(df, column_format="lrrrrrrr", floatfmt="%.2f"):
    """
    Returns a list of LaTeX lines for df that excludes the outer tabular environment
    and top/bottom rules. This lets us manually combine multiple tables into one.
    """
    latex_str = df.to_latex(
        index=False,
        column_format=column_format,
        float_format=floatfmt,
        # You can set booktabs=True if you want \toprule, \midrule, \bottomrule
        # but we'll remove them below anyway.
        escape=False
    )
    lines = latex_str.splitlines()
    
    filtered = []
    for line in lines:
        # Skip the lines that begin/end the tabular environment
        if r"\begin{tabular" in line or r"\end{tabular" in line:
            continue
        # Skip \toprule and \bottomrule
        if r"\toprule" in line or r"\bottomrule" in line:
            continue
        filtered.append(line)
    
    return filtered

# -----------------------------------------------------------------------------
# 3. Combine the three DataFrames into a single LaTeX table with "Panel" labels
# -----------------------------------------------------------------------------
def make_three_panel_table(dfA, dfB, dfC):
    # Adjust column format to match how many columns you have (1 text col + 7 data cols = 8)
    colfmt = "lrrrrrrr"
    
    # Extract the “body” of each DF’s table
    bodyA = df_to_latex_body(dfA, column_format=colfmt)
    bodyB = df_to_latex_body(dfB, column_format=colfmt)
    bodyC = df_to_latex_body(dfC, column_format=colfmt)

    # Start the final table environment
    latex_out = [
        r"\begin{table}[ht]",
        r"\centering",
        r"\caption{Example of Three-Panel Table}",
        r"\label{tab:three_panel}",
        rf"\begin{{tabular}}{{{colfmt}}}",
        r"\toprule"
    ]

    # Panel A
    latex_out.append(r"\multicolumn{8}{c}{\textbf{Panel A: CAPM}} \\")
    latex_out.append(r"\midrule")
    latex_out.extend(bodyA)
    latex_out.append(r"\midrule")

    # Panel B
    latex_out.append(r"\multicolumn{8}{c}{\textbf{Panel B: FF3}} \\")
    latex_out.append(r"\midrule")
    latex_out.extend(bodyB)
    latex_out.append(r"\midrule")

    # Panel C
    latex_out.append(r"\multicolumn{8}{c}{\textbf{Panel C: FF4}} \\")
    latex_out.append(r"\midrule")
    latex_out.extend(bodyC)

    # End the table
    latex_out.append(r"\bottomrule")
    latex_out.append(r"\end{tabular}")
    latex_out.append(r"\end{table}")

    # Join everything into a single string
    return "\n".join(latex_out)


# -----------------------------------------------------------------------------
# 4. Generate and print (or write to file) the combined LaTeX table
# -----------------------------------------------------------------------------
table_latex = make_three_panel_table(dfA, dfB, dfC)
print(table_latex)


\begin{table}[ht]
\centering
\caption{Example of Three-Panel Table}
\label{tab:three_panel}
\begin{tabular}{lrrrrrrr}
\toprule
\multicolumn{8}{c}{\textbf{Panel A: CAPM}} \\
\midrule
  & L & 2 & 3 & 4 & 5 & H & H-L \\
\midrule
α & -0.88 & -0.12 & 0.49 & 1.33 & 2.19 & 3.19 & 4.07 \\
β & 1.42 & 0.91 & 0.93 & 0.96 & 0.97 & 0.94 & -0.48 \\
MKT & 13.94 & 10.11 & 10.16 & 9.33 & 8.47 & 6.59 & -7.35 \\
R² & 0.82 & 0.79 & 0.84 & 0.85 & 0.86 & 0.82 & 0.09 \\
\midrule
\multicolumn{8}{c}{\textbf{Panel B: FF3}} \\
\midrule
  & L & 2 & 3 & 4 & 5 & H & H-L \\
\midrule
α & -1.82 & -1.54 & -0.22 & 0.97 & 1.43 & 2.59 & 4.41 \\
β & 0.67 & 0.82 & 0.91 & 0.96 & 0.98 & 1.03 & 0.36 \\
MKT & 28.85 & 24.70 & 24.61 & 25.29 & 25.50 & 24.57 & -4.28 \\
SMB & 0.15 & 0.11 & 0.20 & 0.31 & 0.29 & 0.41 & 0.26 \\
HML & 0.40 & 0.32 & 0.15 & -0.07 & -0.14 & -0.19 & -0.59 \\
R² & 0.85 & 0.81 & 0.80 & 0.82 & 0.84 & 0.85 & 0.12 \\
\midrule
\multicolumn{8}{c}{\textbf{Panel C: FF4}} \\
\midrule
  & L & 2 & 3 & 4 & 5 & H & H-L \